# **CS 181 HW4 Problem 1**
---

#### Initialize data and parameters

Consider a specific example of when we have $K = 3$ component Gamma distributions. Let's initialize the initial parameter values for $\theta$ and $\beta_k$ as follows:
$$
\begin{align*}
  \theta_k &=  1/K, \\
  \beta_k & = k/K.
\end{align*}
$$

Note that we usually initialize $\theta$ and $\beta_k$ randomly. However, by fixing the initial $\theta$ and $\beta_k$, EM becomes deterministic which makes debugging (and grading) easier.



In [21]:
import torch
import torch.distributions as ds
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

x = torch.load('data.pt')

K = 3
theta = torch.ones(K) / K
alpha = 5.0
betas = (torch.arange(K) + 1) / K

In [22]:
import numpy as np
from scipy.stats import gamma
x = x.numpy()
theta = theta.numpy()
betas = betas.numpy()

#### **Todo:** implement the E-step (Use expression derived in part 3)

In [23]:
def e_step(theta, betas):
    num_data_points = len(x)
    num_components = len(betas)
    
    q = np.zeros((num_data_points, num_components))

    for n in range(num_data_points):
        for k in range(num_components):
            likelihood = gamma.pdf(x[n], alpha, scale=betas[k])
            q[n, k] = likelihood * theta[k]

    q /= q.sum(axis=1, keepdims=True)

    return q

type(betas)

numpy.ndarray

#### **Todo:** implement the M-step (Use expressions derived in parts 4b and 4c)

In [8]:
def m_step(q):
    theta_hat = (q.sum(axis = 1)).sum(axis = 0)/ len(q)
    
    beta_hats = 0
    temp = 0
    for i in range(len(q)):
        beta_hats += q[i] * x[i]
        temp += q[i] * alpha
        
    return theta_hat, (beta_hats/temp).sum(axis = 0)

Log Likelihood Implemented Below

In [12]:
from scipy.special import logsumexp
def log_px(x, theta, betas):
    log_pxs = gamma.logpdf(x, alpha, scale=1/betas) + np.log(theta)
    
    return logsumexp(log_pxs, axis=1)

def log_likelihood(theta, betas):
    return log_px(x, theta, betas).sum()

#### **Todo:** implement EM algorithm

In [14]:
def run_em(theta, betas, iterations=1000):

    prev_log_likelihood = log_likelihood(theta, betas)

    for i in range(0,iterations):

        q = e_step(theta, betas)
        theta, betas = m_step(q)

        new_log_likelihood = log_likelihood(theta, betas)

        if abs(new_log_likelihood - prev_log_likelihood) < 1e-6:
            print("EM algorithm converged ")
            break

        prev_log_likelihood = new_log_likelihood

    return theta, betas

#### Plot

In [ ]:
def make_plot(theta, betas):
    x_test = torch.linspace(0.01, x.max(), 1000)
    prob = log_px(x_test.unsqueeze(-1), theta, betas).exp()
    # prob = np.exp(log_px(x_test.unsqueeze(-1), theta, betas))  # use this line for numpy
    ll = log_likelihood(theta, betas)
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 3))
    
    fig.subplots_adjust(top=0.7)
    fig.suptitle(f'theta = {theta}\nbeta = {betas}\nlog likelihood = {ll:.3e}')

    ax1.set_title('Dataset')
    ax1.hist(x.T, bins=100, color='tomato')
    # ax1.hist(x, bins=100, color='tomato')  # use this line for numpy
    ax2.set_title('Gamma mixture')
    ax2.plot(x_test, prob, color='tomato')
    plt.savefig('p1.pdf', bbox_inches='tight')

In [24]:
theta, betas = run_em(theta, betas)
make_plot(theta, betas)

TypeError: object of type 'numpy.float64' has no len()